# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
weather_df = pd.read_csv("../WeatherPy/weather_df.csv")
weather_df


,Unnamed: 0,City,Country,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph),Date
0,0,tuatapere,NZ,-46.1333,167.6833,25.0,88.0,48.18,2.84,1.652595e+09
1,1,manokwari,ID,-0.8667,134.0833,61.0,75.0,82.96,3.80,1.652595e+09
2,2,sitka,US,57.0531,-135.3300,75.0,76.0,46.29,5.75,1.652595e+09
3,3,butterworth,MY,5.3991,100.3638,20.0,70.0,91.44,10.36,1.652595e+09
4,4,vaini,TO,-21.2000,-175.2000,100.0,59.0,73.92,14.36,1.652595e+09
...,...,...,...,...,...,...,...,...,...,...
626,626,praia da vitoria,PT,38.7333,-27.0667,40.0,82.0,57.76,15.23,1.652595e+09
627,627,khairpur nathan shah,PK,27.0936,67.7337,0.0,9.0,109.38,3.76,1.652595e+09
628,628,katsuura,JP,35.1333,140.3000,100.0,73.0,63.50,16.20,1.652595e+09
629,629,linxia,CN,35.6003,103.2064,49.0,40.0,57.65,8.14,1.652595e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Convert Humidity to float and store
# Also, handle NaN values
weather_df = weather_df.dropna()
humidity = weather_df["Humidity (%)"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = weather_df[["Latitude", "Longitude"]].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidity (%)"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
filtered_weather_df = weather_df

# A max temperature lower than 80 degrees but higher than 70.
filtered_weather_df = filtered_weather_df.loc[(filtered_weather_df["Max Temp (F)"] < 80) & (filtered_weather_df["Max Temp (F)"] > 70)]

# Wind speed less than 10 mph.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Wind Speed (mph)"] < 10]

# Zero cloudiness.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Cloudiness (%)"] == 0]

# Drop any rows with null values
filtered_weather_df = filtered_weather_df.dropna()

filtered_weather_df

,Unnamed: 0,City,Country,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph),Date
177,177,saint george,US,37.1041,-113.5841,0.0,15.0,75.94,5.75,1.652595e+09
318,318,isla mujeres,MX,21.2311,-86.7310,0.0,68.0,75.25,7.63,1.652595e+09
323,323,termiz,UZ,37.2242,67.2783,0.0,34.0,78.89,2.30,1.652595e+09
344,344,laguna,US,38.4210,-121.4238,0.0,59.0,73.54,6.91,1.652595e+09
354,354,makaha,US,21.4694,-158.2175,0.0,68.0,76.91,0.00,1.652595e+09
372,372,maxixe,MZ,-23.8597,35.3472,0.0,66.0,73.74,7.38,1.652595e+09
382,382,namibe,AO,-15.1961,12.1522,0.0,62.0,72.14,2.82,1.652595e+09
393,393,stephenville,US,32.2207,-98.2023,0.0,72.0,77.79,4.61,1.652595e+09
437,437,long beach,US,33.7670,-118.1892,0.0,52.0,78.01,3.44,1.652595e+09
487,487,socorro,US,31.6546,-106.3033,0.0,10.0,78.37,3.44,1.652595e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
hotel_df = filtered_weather_df

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 177: saint george.
Closest hotel is Best Western Plus Abbey Inn.
------------
Retrieving Results for Index 318: isla mujeres.
Closest hotel is Hotel La Joya Isla Mujeres.
------------
Retrieving Results for Index 323: termiz.
Closest hotel is Hotel Ulugbek.
------------
Retrieving Results for Index 344: laguna.
Closest hotel is Holiday Inn Express & Suites Elk Grove Central - Hwy 99, an IHG Hotel.
------------
Retrieving Results for Index 354: makaha.
Closest hotel is Hawaiian Princess Resort.
------------
Retrieving Results for Index 372: maxixe.
Closest hotel is Oceano.
------------
Retrieving Results for Index 382: namibe.
Closest hotel is Hotel Chik Chik Namibe.
------------
Retrieving Results for Index 393: stephenville.
Closest hotel is La Quinta Inn & Suites by Wyndham Stephenville.
------------
Retrieving Results for Index 437: long beach.
Closest hotel is The Westin Long Beach.
------------
Retrieving Results for Index 487: socorro.
Closest hotel i

In [14]:
hotel_df

,Unnamed: 0,City,Country,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph),Date,Hotel Name
177,177,saint george,US,37.1041,-113.5841,0.0,15.0,75.94,5.75,1.652595e+09,Best Western Plus Abbey Inn
318,318,isla mujeres,MX,21.2311,-86.7310,0.0,68.0,75.25,7.63,1.652595e+09,Hotel La Joya Isla Mujeres
323,323,termiz,UZ,37.2242,67.2783,0.0,34.0,78.89,2.30,1.652595e+09,Hotel Ulugbek
344,344,laguna,US,38.4210,-121.4238,0.0,59.0,73.54,6.91,1.652595e+09,Holiday Inn Express & Suites Elk Grove Central...
354,354,makaha,US,21.4694,-158.2175,0.0,68.0,76.91,0.00,1.652595e+09,Hawaiian Princess Resort
372,372,maxixe,MZ,-23.8597,35.3472,0.0,66.0,73.74,7.38,1.652595e+09,Oceano
382,382,namibe,AO,-15.1961,12.1522,0.0,62.0,72.14,2.82,1.652595e+09,Hotel Chik Chik Namibe
393,393,stephenville,US,32.2207,-98.2023,0.0,72.0,77.79,4.61,1.652595e+09,La Quinta Inn & Suites by Wyndham Stephenville
437,437,long beach,US,33.7670,-118.1892,0.0,52.0,78.01,3.44,1.652595e+09,The Westin Long Beach
487,487,socorro,US,31.6546,-106.3033,0.0,10.0,78.37,3.44,1.652595e+09,guarderia imss papalote


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [24]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)

# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)


In [25]:
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))